In [1]:
import pdfplumber
import pandas as pd
import numpy as np
import os
import textEx
from datetime import datetime

In [2]:
def readAllHongTaiPDF(dir:str) -> pd.DataFrame:
    column_names = ['规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位' ]
    all_pages_df = pd.DataFrame(columns=column_names)
    all_pages_df.insert(4, 'date',None) 
    all_pages_df.insert(9, 'date2',None)  
    all_pages_df.insert(14, 'date3', None)  
    all_pages_df.insert(19, 'date4', None)  
    all_pages_df.insert(24, 'date5', None)  

    fail_df=[]
    for filename in os.listdir(dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(dir, filename)
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        try:
                            df = pd.DataFrame(table[1:], columns=column_names)
                            date=filename.replace("詳細報價單","").replace(".pdf","")
                            df.insert(4, 'date', date)  
                            df.insert(9, 'date2', date)  
                            df.insert(14, 'date3', date)  
                            df.insert(19, 'date4', date) 
                            df.insert(24, 'date5', date)  
                        except:
                            df=pd.DataFrame(table[1:])
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
                            fail_df.append(filename)
    df1 = all_pages_df.iloc[:, :5]
    df2 = all_pages_df.iloc[:, 5:10]
    df3 = all_pages_df.iloc[:, 10:15]
    df4 = all_pages_df.iloc[:, 15:20]
    df5 = all_pages_df.iloc[:, 20:25]
    df2.columns = df1.columns
    df3.columns = df1.columns
    df4.columns = df1.columns
    df5.columns = df1.columns
    df_concatenated_corrected = pd.concat([df1, df2, df3,df4,df5], axis=0).reset_index(drop=True)
    df_concatenated_corrected=df_concatenated_corrected.dropna()
    df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
    df_cleaned = df.dropna(subset=["貨名"], how='any').reset_index(drop=True)
    df_cleaned = df_cleaned[df_cleaned["仓位"] != "倉"].reset_index(drop=True)
    df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    column_names_new = ['规格', '貨名牌子', '客户价','仓位']
    all_pages_df = pd.DataFrame(columns=column_names_new)
    all_pages_df.insert(4, 'date',None) 
    for filename in set(fail_df):
        filepath = os.path.join(dir, filename)
        with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        try:
                            df = pd.DataFrame(table[1:], columns=column_names_new)
                            date=filename.replace("詳細報價單(新)","").replace(".pdf","")
                            df.insert(4, 'date', date)
                        except:
                            df=pd.DataFrame(table[1:])
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
                            fail_df.append(filename)
    new_column_names = ['规格', '貨名', '客户价', '仓位','date']
    df = all_pages_df.iloc[:, 0:5].rename(columns=dict(zip(all_pages_df.iloc[:, 0:5].columns, new_column_names)))


    dfall = pd.concat([df_cleaned, df], axis=0).reset_index(drop=True)

    return dfall

In [4]:
dfall=readAllHongTaiPDF(r"C:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\quotation\鸿泰")

Error merging data from 21.11.2022.pdf,<Page:1>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:1>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:2>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:3>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:1>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:2>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:3>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:1>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:2>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:3>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:1>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:2>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:3>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:1>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:2>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:3>
Error merging data 

C:\Users\Balle\AppData\Local\Temp\ipykernel_7596\3390486293.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
C:\Users\Balle\AppData\Local\Temp\ipykernel_7596\3390486293.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [17]:
def _specToList(spec :list) -> list:
    spec.sort()

    # Initialize variables with empty strings
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""

    # Assign values from 'spec' to 'spec1' to 'spec6' based on the length of 'spec'
    if len(spec) > 0:
        spec1 = spec[0]
    if len(spec) > 1:
        spec2 = spec[1]
    if len(spec) > 2:
        spec3 = spec[2]
    if len(spec) > 3:
        spec4 = spec[3]
    if len(spec) > 4:
        spec5 = spec[4]
    if len(spec) > 5:
        spec6 = spec[5]

    return [spec1,spec2,spec3,spec4,spec5,spec6]
        
import re
from datetime import datetime

def extract_date(filename):
    # 尝试匹配 YYYYMMDD_hhmmss 格式
    match = re.search(r'\d+_(\d{2})(\d{2})(\d{2})_\d+', filename)
    if match:
        year, month, day = match.groups()
        year = '20' + year  # 假设年份为 20xx 年
        date_str = f"{year}-{month}-{day}"
        return datetime.strptime(date_str, '%Y-%m-%d').date()
    # 匹配 DD.MM.YYYY
    match = re.search(r'(\d{2})\.(\d{1,2})\.(\d{4})', filename)
    if match:
        day, month, year = match.groups()
        date_str = f"{year}-{month.zfill(2)}-{day}"
        return datetime.strptime(date_str, '%Y-%m-%d').date()

    # 尝试匹配 DDMMYYYY 格式
    match = re.search(r'(\d{2})(\d{2})(\d{4})', filename)
    if match:
        day, month, year = match.groups()
        date_str = f"{year}-{month}-{day}"
        return datetime.strptime(date_str, '%Y-%m-%d').date()

    return filename

def getHongTaiQuote(df: pd.DataFrame) -> pd.DataFrame:
    df['specs'] = df['貨名'].apply(textEx.getSpec)
    try:   
        df['brand'] = df['貨名'].apply(textEx.getBrand)
    except:
        df['brand'] = None
    for index, row in df.iterrows():
        if row['brand'] is not None:
            df.at[index, 'productName'] = row['貨名'].replace(row['brand'], '')
        else:
            df.at[index, 'productName'] = row['貨名']
    try:
        df['productTag'] = df['貨名'].apply(textEx.getProduct)
    except:
        df['productTag'] = None
    df['supplier'] = '鸿泰'
    try:
        df['category'] = df['貨名'].apply(textEx.getCategory)
    except:
        df['category'] = None
    df['packing']=df['规格']
    df['origin'] = df['貨名'].apply(textEx.getCountry)
    try:
        df['effectiveDate'] = df['date'].apply(extract_date)
    except:
        df['effectiveDate'] = None
    df[['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6']] = df['specs'].apply(lambda x: pd.Series(_specToList(x)))
    df['price'] = df['客户价'].str.replace('$','').replace(" ","")
    try:
        df['weightUnit'] = df['客户价'].apply(textEx.getWeightUnit)
    except:
        df['weightUnit'] = None
    try:
        df['warehouse']=df['仓位'].apply(textEx.getWarehoue)
    except:
        df['warehouse'] = None
    df['notes']= None

    df = df.explode('warehouse')
    df['warehouse'] = df['warehouse'].astype(str)

    required_columns = [
    'productName', 'productTag', 'supplier', 'category', 'packing',
    'origin', 'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3',
    'spec4', 'spec5', 'spec6', 'price', 'weightUnit', 'warehouse', 'notes'
    ]
    df=df[required_columns]
    return df

In [ ]:
dfout=getHongTaiQuote(dfall)

In [93]:
dfnewout

,productName,productTag,supplier,category,packing,origin,brand,effectiveDate,spec1,spec2,spec3,spec4,spec5,spec6,price,weightUnit,warehouse,notes
0,"西冷 美麗華 4+(不分條子, 2-4條庄)",西冷,鸿泰,BEEF,抄,美國,MINERVA,132023,2-4條庄,,,,,,27.00p,lb,其士,None
1,西冷 美麗華4+ (#2500廠 ) 3條庄,西冷,鸿泰,BEEF,抄,美國,MINERVA,132023,3條庄,,,,,,27.00p,lb,其士,None
2,frigon 西冷,西冷,鸿泰,BEEF,抄,西班牙,FRIGON,132023,,,,,,,26.30p,lb,其士,None
3,frigon 牛林,牛冧,鸿泰,BEEF,抄,None,FRIGON,132023,,,,,,,23.00p,lb,其士,None
4,美麗華 牛柳 留單 41.00p,牛柳,鸿泰,BEEF,抄,美國,MINERVA,132023,,,,,,,41.50p,lb,其士,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,iqf 小胸 留單13.20k,雞柳,鸿泰,CHICKEN,10k,中國,中佳,2023-12-28,IQF,,,,,,13.30k,kg,其士,None
2237,雞扒 300+ 留單7.30p,雞扒,鸿泰,CHICKEN,15k,中國,中佳,2023-12-28,+300G,,,,,,7.40p,lb,其士,None
2237,雞扒 300+ 留單7.30p,雞扒,鸿泰,CHICKEN,15k,中國,中佳,2023-12-28,+300G,,,,,,7.40p,lb,亞洲,None
2238,雞扒 250+,雞扒,鸿泰,CHICKEN,15k,None,盛佳康,2023-12-28,,,,,,,7.40p,lb,其士,None


In [99]:
dfall.to_csv("鸿泰.csv")